# Everything starts here

In [74]:
import numpy as np
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
import utils
import models

## 1. Split train / validation

In [75]:
# do the split only if it has not already been done
if not os.path.exists(utils.dfs_path + '/training_finetuning.pkl') or not os.path.exists(utils.dfs_path + '/validation_finetuning.pkl'):
    # load all train videos (labelled videos)
    all_train_videos = utils.get_train_test_video_names()['train']
    all_train_labels = pd.read_pickle(utils.labels_path)

    # define split
    split = 0.8
    np.random.seed(69)
    train_videos = np.array(all_train_videos)[np.random.choice(len(all_train_videos), int(0.8 * len(all_train_videos)), replace=False)]
    validation_videos = np.setdiff1d(all_train_videos, train_videos, assume_unique=False)
    train_videos.sort()
    validation_videos.sort()

    # create two subdataframes for training and validation
    training_df = all_train_labels.loc[all_train_labels['videoname'].isin(train_videos)]
    validation_df = all_train_labels.loc[all_train_labels['videoname'].isin(validation_videos)]

    training_df.to_pickle(utils.dfs_path + '/training_finetuning.pkl')
    validation_df.to_pickle(utils.dfs_path + '/validation_finetuning.pkl')

## 2. Model construction

In [3]:
model = models.HernitiaModel(model_name = 'mobilenet_v2_lstm', num_classes = utils.num_classes, pretrained = True, skip_lstm = True).to(utils.device)

In [4]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
count_parameters(model)

2329756

## 3. CNN (bottleneck) finetuning

### 3.1 Set training hyperparameters

In [5]:
# data augmentation and normalization for training
# just normalization for validation
data_transforms = {
    'training': transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'validation': transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [6]:
# training parameters
LEARNING_RATE = 0.003
EPOCHS = 3
BATCH_SIZE = 128
MOMENTUM = 0.9
GAMMA = 0.3
STEP_SIZE = 1

In [7]:
# criterion is cross entropy loss
criterion = nn.CrossEntropyLoss()

# observe that all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)

# decay LR by a factor GAMMA every STEP_SIZE epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

In [8]:
# create pytorch datasets
datasets = {x: utils.HernitiaDataset(utils.dfs_path + '/' + x + '.pkl', data_transforms[x])  for x in ['training', 'validation']}

In [9]:
# instantiate data loaders
dataloaders = {x: utils.DataLoader(dataset=datasets[x], batch_size=BATCH_SIZE, shuffle=True) for x in ['training', 'validation']}

### 3.2 Train

In [ ]:
utils.train_model(model = model, 
                    model_name = model.model_name,  #  name of the model which will be the name of the saved weights file within /weights
                    dataloaders = dataloaders, 
                    criterion = criterion, 
                    optimizer = optimizer, 
                    scheduler = exp_lr_scheduler, 
                    num_epochs=EPOCHS)

## 4. LSTM training

### 4.1 Pad with blank images

In order to process consecutive frames by batch using our dataloaders, each batch loaded should only be composed of frames of a single video. A trick consists of padding each video with blank images at the end.

In [95]:
BATCH_SIZE = 64

if not os.path.exists(utils.dfs_path + '/training_lstm.pkl') or not os.path.exists(utils.dfs_path + '/validation_lstm.pkl'):
    for phase in ['training', 'validation']:
        df = pd.read_pickle(utils.dfs_path + '/' + phase + '_finetuning.pkl')
        for videoname in df['videoname'].unique():
            # pad with blank images at the end of each video
            # so as to only have number of frames multiple of BATCH_SIZE
            # we can then process through the LSTM by batch without shuffling
            num_frames = df[df.videoname == videoname].shape[0]
            num_rows_to_add = (BATCH_SIZE - (num_frames % BATCH_SIZE)) % BATCH_SIZE
            template_white_row = {'videoname': videoname, 'frame': 10000, 'label': -1}
            white_rows_to_add = pd.DataFrame([template_white_row for _ in range(num_rows_to_add)])
            df = pd.concat([df, white_rows_to_add], ignore_index=True)
        df['sort'] = df['videoname'].str[-3:].astype(int)
        # sort rows
        df.sort_values(['sort', 'frame'],inplace=True, ascending=True)
        df = df.drop('sort', axis=1)
        # save df
        df.to_pickle(utils.dfs_path + '/' + phase + '_lstm.pkl')

## 5. Make Kaggle prediction

In [2]:
# create and save testing df
utils.save_testing_df()

'testing dataframe already in storage'

In [3]:
# rebuild model
model = models.mobilenet_v2(pretrained=True).to(utils.device)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, utils.num_classes, device = utils.device)
model_name = 'mobilenet_v2'

# data augmentation and normalization for training
testing_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [4]:
utils.predict_kaggle(model = model, 
                    model_name = model_name, # name of the model from which to load the weights within weights/
                    transform = testing_transforms, 
                    predictions_name = model_name,
                    batch_size = 64) # name of the csv file to which the predictions are saved within predictions/

184003/184003: [===============================>] - ETA 0.7ssss
